# Inicialização e Carregamento

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from itertools import cycle
import fastparquet

In [4]:
pessoas_df = pd.read_csv("people-v2_1E6.csv",sep=";")

In [5]:
pessoas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   document     1000000 non-null  object 
 1   name         1000000 non-null  object 
 2   job          900085 non-null   object 
 3   jobArea      1000000 non-null  object 
 4   jobType      899811 non-null   object 
 5   phoneNumber  1000000 non-null  object 
 6   birthDate    1000000 non-null  object 
 7   city         799999 non-null   object 
 8   state        1000000 non-null  object 
 9   totalSpent   990225 non-null   float64
dtypes: float64(1), object(9)
memory usage: 76.3+ MB


## Data Quality

### Homogenizar e Higienizar coluna document

In [13]:
pessoas_df['document'] = pessoas_df['document'].replace('\.', '', regex=True)
pessoas_df['document'] = pessoas_df['document'].replace('-', '', regex=True)
pessoas_df['document'] = pessoas_df['document'].replace('/', '', regex=True)

### Detectar através da coluna `document` se o registro é de uma Pessoa Física ou Pessoa Jurídica, adicionando uma coluna com essa informação

In [18]:
def calculate_person_type(string):
    if len(string) == 11:
        return "PF"
    
    return "PJ"

In [19]:
pessoas_df['person_type'] = pessoas_df['document'].apply(calculate_person_type)

### Higienizar e homogenizar o formato da coluna `birthDate`: (para YYYY-MM-DD)

In [32]:
def substitui_caracteres(df, coluna, map_dict):
    for item in map_dict:
        df[coluna] = df[coluna].replace(to_replace=item, value=map_dict[item], regex=True)

In [33]:
mapeamento_meses = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
                    'Jul': '07', 'Ago': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [34]:
substitui_caracteres(pessoas_df, 'birthDate', mapeamento_meses)

In [36]:
mapeamento_dias = {'Mon, ': '', 'Tue, ': '', 'Wed, ': '', 'Thu, ': '', 'Fri, ': '', 'Sat, ': '', 'Sun, ': ''}

In [37]:
substitui_caracteres(pessoas_df, 'birthDate', mapeamento_dias)

In [38]:
pessoas_df['birthDate']

0         20-05-1972
1         10-06-1982
2         05/16/1968
3          19810417,
4         04/07/1980
             ...    
999995    16-04-1976
999996    05.22.1981
999997    23-01-1988
999998    04.11.1968
999999    11/27/1988
Name: birthDate, Length: 1000000, dtype: object

In [39]:
pessoas_df['birthDate'] = pd.to_datetime(pessoas_df['birthDate'])

In [41]:
pessoas_df['birthDate']

0        1972-05-20
1        1982-10-06
2        1968-05-16
3        1981-04-17
4        1980-04-07
            ...    
999995   1976-04-16
999996   1981-05-22
999997   1988-01-23
999998   1968-04-11
999999   1988-11-27
Name: birthDate, Length: 1000000, dtype: datetime64[ns]

### Existem duas colunas nesse dataset que alguns registros estão trocadas. Quais são essas colunas? 

## Agregação dos dados

### Quais são as 5 PF que mais gastaram (`totalSpent`)? 

In [65]:
pessoas_fisicas_df = pessoas_df.where(pessoas_df['person_type'] == 'PF')

In [74]:
spent_ordenado_df = pessoas_fisicas_df.sort_values(by='totalSpent', ascending=False)

In [76]:
spent_ordenado_df

,document,name,job,jobArea,jobType,phoneNumber,birthDate,city,state,totalSpent,person_type
968481,82190864755,Euvanderson Costa,Supervisor Branding Sênior,Branding,Desenvolvedor,(34) 63123-5050,1965-10-25,Grande Kacia,Pernambuco,1000.00,PF
464764,18741688813,Sra. Rocio Martins,Especialista Mobilidade Sênior,Configuração,Supervisor,(45) 9143-5347,1985-06-06,Vania do Descoberto,PA,1000.00,PF
280776,61454551445,Wandir Martins,NaN,Otimização,Orquestrador,+55 (74) 2431-7486,1988-11-09,Martins de Nossa Senhora,AL,999.99,PF
4807,58953147670,Valeria Souza,Gerente Paradigma Chefe,Contas,Planejador,(57) 3724-5403,1970-09-02,Ligeria do Descoberto,Alagoas,999.99,PF
233482,17195012700,Regina Melo,Especialista Identidade Frente,Mobilidade,Desenvolvedor,Implementação,1962-07-18,Martins do Sul,MS,999.99,PF
...,...,...,...,...,...,...,...,...,...,...,...
999989,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
999990,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
999992,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
999996,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
